In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
#import dependencies
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

In [ ]:
xls = pd.ExcelFile('/kaggle/input/Data.xlsx')

# read every sheet in the Excel file
data = pd.read_excel(xls, 'Description') # description of the data
prodbal = pd.read_excel(xls, 'Products_ActBalance') # product account balance data set
socdem = pd.read_excel(xls, 'Soc_Dem') # social demography data set
inout = pd.read_excel(xls, 'Inflow_Outflow') # behavior as cash inflow and outflow
sales = pd.read_excel(xls, 'Sales_Revenues') # sales and revenue

# train set
df = socdem.merge(sales, on='Client', how='inner')
df1 = df.merge(prodbal, on='Client', how='inner')
X = df1.merge(inout, on='Client', how='left') # left join

# counter variables such as count of CA, SA, MF, OVD, CC, and CL, its NaN value will be replaced by 0.
counters = ['Count_CA', 'Count_SA', 'Count_MF', 'Count_OVD', 'Count_CC', 'Count_CL']
X[counters] = X[counters].fillna(value=0)

# 6 targets
MF_y = X['Sale_MF']
CC_y = X['Sale_CC']
CL_y = X['Sale_CL']
rMF_y = X['Revenue_MF']
rCC_y = X['Revenue_CC']
rCL_y = X['Revenue_CL']
X.drop(['Client','Sale_MF', 'Sale_CC', 'Sale_CL', 'Revenue_MF', 'Revenue_CC', 'Revenue_CL'], axis=1, inplace=True)

# test set
df2 = socdem.merge(prodbal, on='Client', how='inner')
tdata = df2.merge(inout, on='Client', how='left') # left join
test = tdata[~tdata.Client.isin(sales.Client)]

# convert categorical variables to numerical with cardinality less than 10
low_cardinality_cols = [cname for cname in X.columns if 
                                X[cname].nunique() < 10 and
                                X[cname].dtype == "object"]

# make sure all data are positive
X = pd.get_dummies(X, columns=low_cardinality_cols)
X = X.abs()
test = pd.get_dummies(test, columns=low_cardinality_cols)
test = test.abs()

# clean data 
X.fillna(0, inplace=True)
test.fillna(0, inplace=True)
X = X.replace('\n','', regex=True)
test = test.replace('\n','', regex=True)

## Sales

### Mutual Fund Sales

In [201]:
# split the data set
MF_X_train, MF_X_valid, MF_y_train, MF_y_valid = train_test_split(X, MF_y, test_size=0.25, random_state=1)

# scale train set to be fed to feature selection
sc = StandardScaler()
MF_X_train_scaled = sc.fit_transform(MF_X_train)

# feature selection
MF_feat = LogisticRegression()
rfe = RFE(MF_feat, 2)
fit = rfe.fit(MF_X_train_scaled, MF_y_train)

# extract best features
MF_features = [i for i, x in enumerate(fit.ranking_) if x == True]
MF_F = list(X.columns[MF_features])

# training and validation set with selected features
MF_X_train = MF_X_train[MF_F]
MF_X_valid = MF_X_valid[MF_F]

In [202]:
# random forest classifier
MF_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)

MF_model.fit(MF_X_train, MF_y_train)
MF_preds = MF_model.predict(MF_X_valid)

print("Mutual funds sales prediction accuracy: ")
print(accuracy_score(MF_preds, MF_y_valid))

Mutual funds sales prediction accuracy: 
0.8148148148148148


In [ ]:
# finding the best hyperparameter for randomforest classifier in mutual funds sales
parameters = [{'n_estimators':[50, 100, 150, 200, 250, 300, 350, 400, 450, 500], 'max_depth': range(5, 10)}]
grid_search = GridSearchCV(estimator=MF_model, param_grid=parameters, scoring='accuracy', cv=10, n_jobs=-1)
grid = grid_search.fit(MF_X_train, MF_y_train)
print(grid_search.best_params_)

### Credit Card Sales

In [203]:
# split the data set
CC_X_train, CC_X_valid, CC_y_train, CC_y_valid = train_test_split(X, CC_y, test_size=0.25, random_state=1)

# scale train set to be fed to feature selection
#sc = StandardScaler()
CC_X_train_scaled = sc.fit_transform(CC_X_train)

# feature selection
CC_feat = LogisticRegression()
rfe = RFE(CC_feat, 2)
fit = rfe.fit(CC_X_train_scaled, CC_y_train)

# extract best features
CC_features = [i for i, x in enumerate(fit.ranking_) if x == True]
CC_F = list(X.columns[CC_features])

# training and validation set with selected features
CC_X_train = CC_X_train[CC_F]
CC_X_valid = CC_X_valid[CC_F]

In [220]:
# random forest classifier for credit card sales
CC_model = RandomForestClassifier(n_estimators=150, max_depth=6, random_state=1)

CC_model.fit(CC_X_train, CC_y_train)
CC_preds = CC_model.predict(CC_X_valid)

print("Credit cards sales prediction accuracy: ")
print(accuracy_score(CC_preds, CC_y_valid))

Credit cards sales prediction accuracy: 
0.7695473251028807


In [219]:
# finding the best hyperparameter for randomforest classifier in credit card sales
parameters = [{'n_estimators':[50, 100, 150, 200, 250, 300, 350, 400, 450, 500], 'max_depth': range(5, 10)}]
grid_search = GridSearchCV(estimator=CC_model, param_grid=parameters, scoring='accuracy', cv=10, n_jobs=-1)
grid = grid_search.fit(CC_X_train, CC_y_train)
print(grid_search.best_params_)

{'max_depth': 6, 'n_estimators': 150}


### Consumer Loans Sales

In [205]:
# split the data set
CL_X_train, CL_X_valid, CL_y_train, CL_y_valid = train_test_split(X, CL_y, test_size=0.25, random_state=1)

# scale train set to be fed to feature selection
#sc = StandardScaler()
CL_X_train_scaled = sc.fit_transform(CL_X_train)

# feature selection
CL_feat = LogisticRegression()
rfe = RFE(CL_feat, 2)
fit = rfe.fit(CL_X_train_scaled, CL_y_train)

# extract best features
CL_features = [i for i, x in enumerate(fit.ranking_) if x == True]
CL_F = list(X.columns[CL_features])

# training and validation set with selected features
CL_X_train = CL_X_train[CL_F]
CL_X_valid = CL_X_valid[CL_F]

In [206]:
# random forest classifier for consumer loans sales
CL_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

CL_model.fit(CL_X_train, CL_y_train)
CL_preds = CL_model.predict(CL_X_valid)

print("Consumer loans sales prediction accuracy: ")
print(accuracy_score(CL_preds, CL_y_valid))

Consumer loans sales prediction accuracy: 
0.7448559670781894


In [ ]:
# finding the best hyperparameter for randomforest classifier in consumer loans sales
parameters = [{'n_estimators':[50, 100, 150, 200, 250, 300, 350, 400, 450, 500], 'max_depth': range(5, 10)}]
grid_search = GridSearchCV(estimator=CL_model, param_grid=parameters, scoring='accuracy', cv=10, n_jobs=-1)
grid = grid_search.fit(CL_X_train, CL_y_train)
print(grid_search.best_params_)

## Revenues

### Mutual Funds Revenues

In [207]:
# filter data according to MF sale
MF_rX = X.join(MF_y)
MF_rX = MF_rX[MF_rX['Sale_MF']==1]

# response variable
rMF_y = rMF_y[rMF_y>0] # filter response variable by removing data with no values 

# test set for mutual fund
MF_rtest = test.join(MF_y)

In [208]:
# split the data set
rMF_X_train, rMF_X_valid, rMF_y_train, rMF_y_valid = train_test_split(MF_rX, rMF_y, test_size=0.25, random_state=0)

# scale train set to be fed to feature selection
sc = StandardScaler()
rMF_X_train_scaled = sc.fit_transform(rMF_X_train)


# feature selection for continuous feature response
rMF_feat = LinearRegression()
rfe = RFE(rMF_feat, 3)
fit = rfe.fit(rMF_X_train_scaled, rMF_y_train)

# extract best features
rMF_features = [i for i, x in enumerate(fit.ranking_) if x == True]
rMF_F = list(X.columns[rMF_features])

# training and validation set with selected features
rMF_X_train = rMF_X_train[rMF_F]
rMF_X_valid = rMF_X_valid[rMF_F]

In [209]:
# model mutual funds revenue
rMF_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=1) # Your code here

rMF_model.fit(rMF_X_train, rMF_y_train) # Your code here
rMF_pred = rMF_model.predict(rMF_X_valid)
rMF = rMF_model.predict(MF_rtest[rMF_F])
#print(rMF)

# MAE score
rMF_mae = mean_absolute_error(rMF_pred, rMF_y_valid)
print("Mean Absolute Error:", rMF_mae)

Mean Absolute Error: 11.473126530716785


### Credit Cards Revenues

In [210]:
# filter data according to CC sale
CC_rX = X.join(CC_y)
CC_rX = CC_rX[CC_rX['Sale_CC']==1]

# response variable
rCC_y = rCC_y[rCC_y>0]

# test set for mutual fund
CC_rtest = test.join(CC_y)
#CC_rtest = CC_rtest[CC_rtest['Sale_CC']==1]

In [211]:
# split the data set
rCC_X_train, rCC_X_valid, rCC_y_train, rCC_y_valid = train_test_split(CC_rX, rCC_y, test_size=0.25, random_state=0)

# scale train set to be fed to feature selection
sc = StandardScaler()
rCC_X_train_scaled = sc.fit_transform(rCC_X_train)


# feature selection for continuous feature response
rCC_feat = LinearRegression()
rfe = RFE(rCC_feat, 5)
fit = rfe.fit(rCC_X_train_scaled, rCC_y_train)

# extract best features
rCC_features = [i for i, x in enumerate(fit.ranking_) if x == True]
rCC_F = list(X.columns[rCC_features])

# training and validation set with selected features
rCC_X_train = rCC_X_train[rCC_F]
rCC_X_valid = rCC_X_valid[rCC_F]

In [212]:
# model credit card revenue
rCC_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=1) # Your code here

rCC_model.fit(rCC_X_train, rCC_y_train) # Your code here
rCC_pred = rCC_model.predict(rCC_X_valid)
rCC = rCC_model.predict(CC_rtest[rCC_F])
print(rCC)

# MAE score
rCC_mae = mean_absolute_error(rCC_pred, rCC_y_valid)
print("Mean Absolute Error:", rCC_mae)

[  2.764536     5.185303     5.784325    14.797865     6.167162
   2.190125     3.6149673    4.2012925   19.000998     9.117914
   2.8784282    3.5248792    5.280957     8.033047     2.33139
   4.4424       6.7637334    2.2810974    4.244567     4.266032
   8.750064     3.3708022    1.6206833    3.92156      6.649774
   3.8663442    6.7163863    9.879681     2.5461314    3.787422
  11.64529      4.815945     2.7027595    3.3143957    2.8389766
  11.270875     6.154348     2.8389766   28.250088     5.5115657
   2.3450234    3.1052842    8.062239     3.9652624    6.222304
   4.8475046    7.0131207    2.875147     4.0675774    6.0288997
   3.814079     1.1432714    5.776988     5.1914926    2.5045424
   3.4807189    3.8808196    4.6906486    4.4424      13.889212
   3.9462032    5.90891      6.2121096    3.9614305    4.4163733
  31.031126    11.340098     3.870669     2.095209     4.2964377
  11.599456    11.412265     8.706678    13.030843    12.27338
   2.1836958   65.80337      3.04706

### Consumer Loans Revenues

In [213]:
# filter data according to CC sale
CL_rX = X.join(CL_y)
CL_rX = CL_rX[CL_rX['Sale_CL']==1]

# response variable
rCL_y = rCL_y[rCL_y>0]

# test set for mutual fund
CL_rtest = test.join(CL_y)
#CL_rtest = CL_rtest[CL_rtest['Sale_CL']==1]

In [214]:
# split the data set
rCL_X_train, rCL_X_valid, rCL_y_train, rCL_y_valid = train_test_split(CL_rX, rCL_y, test_size=0.25, random_state=0)

# scale train set to be fed to feature selection
sc = StandardScaler()
rCL_X_train_scaled = sc.fit_transform(rCL_X_train)


# feature selection for continuous feature response
rCL_feat = LinearRegression()
rfe = RFE(rCL_feat, 5)
fit = rfe.fit(rCL_X_train_scaled, rCL_y_train)

# extract best features
rCL_features = [i for i, x in enumerate(fit.ranking_) if x == True]
rCL_F = list(X.columns[rCL_features])

# training and validation set with selected features
rCL_X_train = rCL_X_train[rCL_F]
rCL_X_valid = rCL_X_valid[rCL_F]

In [215]:
# model consumer loans revenue
rCL_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=1) # Your code here

rCL_model.fit(rCL_X_train, rCL_y_train) # Your code here
rCL_pred = rCL_model.predict(rCL_X_valid)
rCL = rCL_model.predict(CL_rtest[rCL_F])
print(rCL)

# MAE score
rCL_mae = mean_absolute_error(rCL_pred, rCL_y_valid)
print("Mean Absolute Error:", rCL_mae)

[ 20.455196   10.464836   15.372995    8.627318    9.542228    8.6606045
   7.8425026  15.398468    7.467892    9.770492   13.381905   13.236541
  16.383804    7.655943   13.998751   12.278206    6.3666806  11.872872
  10.300496   11.175934    8.368822    9.671741   13.809875   13.037234
  11.68983     6.328103   14.887839    8.667537   10.940919    8.892328
   5.760477    9.333761    9.1101885   7.485529   12.009661    7.9353685
   7.5987134  10.101365   10.300181    8.193459    5.8172097  14.092807
  11.097532   13.277887   11.628804    6.4574904   6.5926304  11.538193
   8.446924    8.789594    5.0779486  10.37228    14.154314   13.24354
  10.154699   10.985061    6.8468413  13.278986   11.000316   13.55026
   5.6492624  24.832344   11.070411  130.5818     17.917856   11.421741
  15.046638   15.679743   13.045436   11.231548   12.133832    9.534884
   8.23987    12.896524   10.355021   13.247457   10.762495   12.789636
  11.625763    5.865171   12.339751    9.05127     5.1669273  11

# Results

In [216]:
# sales prediction data frame in terms of probability
MF_preds = MF_model.predict_proba(test[MF_F])
MF_preds = MF_preds[:, 1]

CC_preds = CC_model.predict_proba(test[CC_F])
CC_preds = CC_preds[:, 1]

CL_preds = CL_model.predict_proba(test[CL_F])
CL_preds = CL_preds[:, 1]

sales_output = pd.DataFrame({'Client':test.Client, 'MF_sales':MF_preds, 'CC_sales':CC_preds, 'CL_sales':CL_preds})

# revenues of each product
MF_revenues = pd.DataFrame({'Client':MF_rtest.Client, 'MF_Revenue':rMF})
CC_revenues = pd.DataFrame({'Client':CC_rtest.Client, 'CC_Revenue':rCC})
CL_revenues = pd.DataFrame({'Client':CL_rtest.Client, 'CL_Revenue':rCL})

## Top 100 prospects

Revenues result will be merged based on top 100 clients of predicted sales. And predicted revenue will be computed.

In [ ]:
sales_output['mean'] = sales_output[['MF_sales', 'CC_sales', 'CL_sales']].mean(numeric_only=True, axis=1)
sales_output = sales_output.astype({'Client':int})

# identify which client will take which product
prospects = sales_output.sort_values('mean', ascending=False).head(100)
prospects['prod'] = prospects[['MF_sales', 'CC_sales', 'CL_sales']].idxmax(axis=1) 

In [217]:
# groupby 
mutual_funds = prospects[prospects['prod'] == 'MF_sales']
mutual_funds.reset_index(drop=True, inplace=True)

credit_cards = prospects[prospects['prod'] == 'CC_sales']
credit_cards.reset_index(drop=True, inplace=True)

consumer_loans = prospects[prospects['prod'] == 'CL_sales']
consumer_loans.reset_index(drop=True, inplace=True)

In [218]:
# predicted clients with revenue of mutual fund will be merged to the revenues 646 data result
client_pred_MF_revenue = mutual_funds.merge(MF_revenues, on='Client', how='inner')
client_pred_MF_revenue.rename({'MF_Revenue':'revenue'}, axis=1, inplace=True)
client_pred_CC_revenue = credit_cards.merge(CC_revenues, on='Client', how='inner')
client_pred_CC_revenue.rename({'CC_Revenue':'revenue'}, axis=1, inplace=True)
client_pred_CL_revenue = consumer_loans.merge(CL_revenues, on='Client', how='inner')
client_pred_CL_revenue.rename({'CL_Revenue':'revenue'}, axis=1, inplace=True)

result = pd.concat([client_pred_MF_revenue, client_pred_CC_revenue, client_pred_CL_revenue], sort=False)
result.reset_index(drop=True, inplace=True)

result.head()

,Client,MF_sales,CC_sales,CL_sales,mean,prod,revenue
0,110.0,0.753651,0.067662,0.536398,0.452570,MF_sales,0.964439
1,1093.0,0.731274,0.374956,0.189677,0.431969,MF_sales,7.302523
2,228.0,0.490276,0.485798,0.262196,0.412757,MF_sales,2.793087
3,1097.0,0.479356,0.401709,0.228271,0.369779,MF_sales,5.811006
4,1007.0,0.709202,0.240547,0.156439,0.368729,MF_sales,4.377915


In [221]:
result.revenue.sum()

1033.4861

### Prospected clients for every product given its probability

In [178]:
# mutual funds
prospects_MF = prospects[prospects['MF_sales'] > 0.5]
prospects_MF = prospects_MF.drop(['prod', 'CL_sales', 'CC_sales', 'mean'], axis=1)
prospects_MF.reset_index(drop=True, inplace=True)

# credit cards
prospects_CC = prospects[prospects['CC_sales'] > 0.5]
prospects_CC =prospects_CC.drop(['prod', 'CL_sales', 'MF_sales', 'mean'], axis=1)
prospects_CC.reset_index(drop=True, inplace=True)

# consumer loans
prospects_CL = prospects[prospects['CL_sales'] > 0.5]
prospects_CL =prospects_CL.drop(['prod', 'CC_sales', 'MF_sales', 'mean'], axis=1)
prospects_CL.reset_index(drop=True, inplace=True)

,Client,MF_sales
0,340.0,0.529509
1,110.0,0.753651
2,1093.0,0.731274
3,1007.0,0.709202
4,347.0,0.554868
